In [1]:
#Cleaning the Data
import pandas as pd 
import numpy as np
import statsmodels.api as statmod 
import scipy.stats as stats
from scipy.optimize import minimize 
import scipy.optimize as opt
data1 = pd.read_stata('PS4_data.dta') 
'''
hlabinc = annual labor income of the head
hannhrs = annual hours of the head
hsex = gender of the head (1 = Male, 2 = Female)
hrace = race of the head (1 = white, 2 = Black, 3 = Native American, 4 = Asian/Pacific Islander, 5 = Hispanic, 6,7 = Other)
age = age of the head
hyrsed = years of education of the head
'''
data1["age squared"]=data1["age"]*data1["age"]
print("Data Statistics:")
data1.describe()

Data Statistics:


,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce,age squared
count,123786.000000,123786.000000,123786.000000,123786.000000,123786.000000,9.023300e+04,48496.000000,123786.000000,90603.000000,123656.000000,...,123786.000000,80758.0,123786.000000,80758.000000,123786.000000,80758.000000,122809.000000,80091.000000,123786.000000,123786.000000
mean,1494.639475,1984.831273,3271.379429,1679.269897,633.026917,4.211505e+04,22026.289062,0.843771,1.098220,1.129731,...,1.233072,2.0,45.545547,41.390785,39.620201,55.346169,12.666091,12.720081,0.557690,2384.941162
std,838.901790,9.836212,2277.056058,1061.704712,878.422791,4.670424e+04,21336.107422,1.182829,0.356161,0.394627,...,0.422940,0.0,17.623671,14.786721,69.003265,77.864296,2.917721,2.422607,0.265198,1814.390625
min,1.000000,1967.000000,1.000000,0.000000,0.000000,6.353981e-01,1.192780,0.000000,1.000000,1.000000,...,1.000000,2.0,16.000000,13.000000,1.000000,1.000000,1.000000,1.000000,0.000000,256.000000
25%,772.000000,1977.000000,1444.000000,832.000000,0.000000,1.979858e+04,8016.247070,0.000000,1.000000,1.000000,...,1.000000,2.0,31.000000,29.000000,1.000000,2.000000,12.000000,12.000000,0.362158,961.000000
50%,1517.000000,1985.000000,2984.000000,1976.000000,0.000000,3.460022e+04,18122.412109,0.000000,1.000000,1.000000,...,1.000000,2.0,42.000000,39.000000,3.000000,3.000000,12.000000,12.000000,0.599887,1764.000000
75%,2224.000000,1993.000000,4763.000000,2350.000000,1454.000000,5.267309e+04,30256.060547,2.000000,1.000000,1.000000,...,1.000000,2.0,58.000000,51.000000,22.000000,170.000000,15.000000,14.000000,0.786908,3364.000000
max,2930.000000,2002.000000,16968.000000,7800.000000,5840.000000,3.771521e+06,856942.062500,11.000000,8.000000,8.000000,...,2.000000,2.0,102.000000,95.000000,227.000000,231.000000,17.000000,17.000000,0.928007,10404.000000


In [2]:
data2 = data1.dropna(how = 'any', subset = ['hlabinc', 'hannhrs', 'hsex', 'hrace', 'age','age squared', 'hyrsed']) #drop missing values
data3 = data2[['id68', 'year', 'intid', 'hlabinc', 'hannhrs', 'hsex', 'hrace', 'age','age squared','hyrsed']]
data3['ahrs'] = data3['hannhrs'].where(data3['hannhrs']>0)
data3['hrw'] = data3['hlabinc']/data3['ahrs'] 
data3 = data3[(data3.hsex == 1.0) & (data3.age >= 25) & (data3.age <= 60) & (data3.hrw > 7)] # select male heads of HH whose age is between 25 & 60 (included!), and wage > $7/hr
data3['ln_hrw'] = np.log(data3['hrw']) 
race_dummy = pd.get_dummies(data3['hrace']) # Defining race dummies; no Hispanic dummy is created because there is no Hispanic individual in data
data3['constant'] = 1
data = pd.concat([data3, race_dummy], axis = 1) 
data.rename(columns = {1.0: 'White', 2.0: 'Black', 3.0: 'Others'}, inplace = True) 
# Separate years for estimation
data1971 = data[data['year'] == 1971]
data1980 = data[data['year'] == 1980]
data1990 = data[data['year'] == 1990]
data2000 = data[data['year'] == 2000]
data.describe()

<ipython-input-2-7924cda79cc5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['ahrs'] = data3['hannhrs'].where(data3['hannhrs']>0)
<ipython-input-2-7924cda79cc5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['hrw'] = data3['hlabinc']/data3['ahrs']


,id68,year,intid,hlabinc,hannhrs,hsex,hrace,age,age squared,hyrsed,ahrs,hrw,ln_hrw,constant,White,Black,Others
count,57062.000000,57062.000000,57062.000000,5.706200e+04,57062.000000,57062.0,57062.000000,57062.000000,57062.000000,57062.000000,57062.000000,57062.000000,57062.000000,57062.0,57062.000000,57062.000000,57062.000000
mean,1507.470558,1986.575672,3480.375311,5.282805e+04,2228.557617,1.0,1.101416,39.243629,1631.315552,13.529967,2228.557617,24.320921,3.010804,1.0,0.921103,0.056377,0.022519
std,828.361439,8.712311,2253.229068,5.236477e+04,620.054077,0.0,0.369015,9.578858,791.953796,2.450013,620.054077,25.209909,0.544096,0.0,0.269580,0.230651,0.148367
min,1.000000,1971.000000,1.000000,1.666980e+01,2.000000,1.0,1.000000,25.000000,625.000000,1.000000,2.000000,7.000252,1.945946,1.0,0.000000,0.000000,0.000000
25%,782.000000,1979.000000,1690.000000,3.037345e+04,1952.000000,1.0,1.000000,31.000000,961.000000,12.000000,1952.000000,13.950662,2.635527,1.0,1.000000,0.000000,0.000000
50%,1542.000000,1987.000000,3296.000000,4.382858e+04,2160.000000,1.0,1.000000,38.000000,1444.000000,13.000000,2160.000000,19.914677,2.991457,1.0,1.000000,0.000000,0.000000
75%,2225.000000,1994.000000,5002.000000,6.138495e+04,2519.000000,1.0,1.000000,47.000000,2209.000000,16.000000,2519.000000,27.790929,3.324710,1.0,1.000000,0.000000,0.000000
max,2930.000000,2002.000000,16968.000000,3.771521e+06,5840.000000,1.0,3.000000,60.000000,3600.000000,17.000000,5840.000000,1717.330322,7.448526,1.0,1.000000,1.000000,1.000000


In [4]:
#MLE
'''
ln(w_it) = alpha + beta_1 * Educ_it + beta_2 * Age_it +beta_3 * Age squared_it+ beta_4 * Black_it + beta_5 * Hispanic_it + beta_6 * OtherRace_it + epsilon_it
w_it = wage of individual i in survey year t
Educ_it = education in years
Age_it = age in years
Age squared_it= age squared in years
Black_it, Hispanic_it, OtherRace_it = dummy variables for race = Black, Hispanic, Not (belongs to {White, Black, Hispanic})
'''
def Log_Lik(params, t): #define negative maximum likelihood
    # Coeff.s
    beta0, beta1, beta2, beta3, beta4, beta5, sigma = params
    beta = np.array([beta0, beta1, beta2, beta3, beta4, beta5])
    n = len(t)
    # Independent variables matrix (No Hispanic)
    x0 = np.array(t['constant']).astype('float')
    x1 = np.array(t['hyrsed']).astype('float')
    x2 = np.array(t['age']).astype('float')
    x3 = np.array(t['age squared']).astype('float')
    x4 = np.array(t['Black']).astype('float')
    x5 = np.array(t['Others']).astype('float')
    X = np.column_stack((x0, x1, x2, x3, x4, x5))
    # Dependent variable matrix
    y = np.array(t['ln_hrw']).astype('float')
    y_bar = np.dot(X, beta)
    ll = (-(n/2)*np.log(2*np.pi) - (n/2)*np.log(sigma**2) - (1/(2*sigma**2))*((y-y_bar).T @ (y - y_bar)))
    return (-ll)

In [65]:
# MLE; 'Nelder-Mead'
b0 = 0.6
b1 = 0.06
b2 = 0.06
b3 = 0.001
b4 = 0.15
b5 = 0.01
SD = 0.5
beta = [b0, b1, b2, b3, b4, b5]
params = [b0, b1, b2, b3, b4, b5, SD]
bounds = ((1e-10, None), (None,None), (None,None), (None,None), (None,None), (None,None), (None, None))

print("Coefficients: ", "[b0  b1  b2  b3  b4  b5  SD]")
print("Whole Data: ", opt.minimize(Log_Lik, params, args=(data), method='Nelder-Mead', bounds=bounds).x)
print("year = 1971: ", opt.minimize(Log_Lik, params, args=(data1971), method='Nelder-Mead', bounds=bounds).x)
print("year = 1980: ", opt.minimize(Log_Lik, params, args=(data1980), method='Nelder-Mead', bounds=bounds).x)
print("year = 1990: ", opt.minimize(Log_Lik, params, args=(data1990), method='Nelder-Mead', bounds=bounds).x)
print("year = 2000: ", opt.minimize(Log_Lik, params, args=(data2000), method='Nelder-Mead', bounds=bounds).x)

Coefficients:  [b0  b1  b2  b3  b4  b5  SD]
Whole Data:  [ 0.53047904  0.07642193  0.06022729 -0.00055677 -0.16231726  0.02204344
  0.48715692]
year = 1971:  [ 0.58967095  0.0664388   0.06485238 -0.0006166  -0.16330773 -0.02399601
  0.40588664]
year = 1980:  [ 9.98478251e-01  6.61335024e-02  4.56168227e-02 -4.00085835e-04
 -9.94140469e-02  5.82350847e-02  4.47848527e-01]
year = 1990:  [ 0.27135664  0.09550907  0.05812985 -0.0005434  -0.16776687 -0.01546005
  0.48150638]
year = 2000:  [-0.32542844  0.11057272  0.0854458  -0.00089813 -0.24626635  0.05061187
  0.53453446]


In [66]:
# MLE; 'L-BFGS-B'
b0 = 0.6
b1 = 0.06
b2 = 0.06
b3 = 0.001
b4 = 0.15
b5 = 0.01
SD = 0.5
beta = [b0, b1, b2, b3, b4, b5]
params = [b0, b1, b2, b3, b4, b5, SD]
bounds = ((1e-10, None), (None,None), (None,None), (None,None), (None,None), (None,None), (None, None))


print("Coefficients: ", "[b0  b1  b2  b3  b4  b5  SD]")
print("Whole dataset: ", opt.minimize(Log_Lik, params, args=(data), method='L-BFGS-B', bounds=bounds).x)
print("year = 1971: ", opt.minimize(Log_Lik, params, args=(data1971), method='L-BFGS-B', bounds=bounds).x)
print("year = 1980: ", opt.minimize(Log_Lik, params, args=(data1980), method='L-BFGS-B', bounds=bounds).x)
print("year = 1990: ", opt.minimize(Log_Lik, params, args=(data1990), method='L-BFGS-B', bounds=bounds).x)
print("year = 2000: ", opt.minimize(Log_Lik, params, args=(data2000), method='L-BFGS-B', bounds=bounds).x)

Coefficients:  [b0  b1  b2  b3  b4  b5  SD]
Whole dataset:  [ 5.92005799e-01  7.60780094e-02  5.73959604e-02 -5.23312722e-04
 -1.63278373e-01  1.70184809e-02  4.87163123e-01]
year = 1971:  [ 0.59503445  0.06645817  0.0644871  -0.00061204 -0.16396128  0.02389293
  0.40585189]
year = 1980:  [ 6.18058995e-01  6.79054574e-02  6.36749273e-02 -6.13240877e-04
 -9.87568969e-02  1.74516624e-02  4.48286295e-01]
year = 1990:  [ 5.78117173e-01  9.39753664e-02  4.39933377e-02 -3.76221574e-04
 -1.73485234e-01 -4.46351522e-03  4.81759471e-01]
year = 2000:  [ 5.53691199e-01  1.03694088e-01  4.67201806e-02 -4.40214834e-04
 -2.66201369e-01 -2.57408840e-02  5.35794035e-01]


In [81]:
# MLE; 'SLSQP'
b0 = 0.6
b1 = 0.06
b2 = 0.06
b3 = 0.001
b4 = 0.2
b5 = 0.01
SD = 5.0
beta = [b0, b1, b2, b3, b4, b5]
params = [b0, b1, b2, b3, b4, b5, SD]
bounds = ((1e-10, None), (None,None), (None,None), (None,None), (None,None), (None,None), (None, None))
print("Coefficients: ", "[b0  b1  b2  b3  b4  b5  SD]")
print("Whole dataset: ", opt.minimize(Log_Lik, params, args=(data), method='SLSQP', bounds=bounds).x)
print("year = 1971: ", opt.minimize(Log_Lik, params, args=(data1971), method='SLSQP', bounds=bounds).x)
print("year = 1980: ", opt.minimize(Log_Lik, params, args=(data1980), method='SLSQP', bounds=bounds).x)
print("year = 1990: ", opt.minimize(Log_Lik, params, args=(data1990), method='SLSQP', bounds=bounds).x)
print("year = 2000: ", opt.minimize(Log_Lik, params, args=(data2000), method='SLSQP', bounds=bounds).x)

Coefficients:  [b0  b1  b2  b3  b4  b5  SD]
Whole dataset:  [ 0.52409241  0.07640094  0.06058532 -0.00056109 -0.16266545  0.00847067
 -0.48714831]
year = 1971:  [ 0.585529    0.06648572  0.06496607 -0.00061777 -0.16409956  0.01779111
  0.40586387]
year = 1980:  [ 9.97979197e-01  6.59910191e-02  4.58005345e-02 -4.02243037e-04
 -1.03036552e-01  1.23016751e-02  4.47801138e-01]
year = 1990:  [ 0.2723721   0.09548689  0.05812143 -0.00054328 -0.1679651  -0.05195671
  0.48149014]
year = 2000:  [ 1.00058703e-10  1.07937833e-01  7.13919319e-02 -7.32660027e-04
 -2.60237376e-01 -6.33320876e-02 -5.34352476e-01]


In [83]:
# Comparing the MLE coefficients for 3 different methods:
print("Whole Data; Nelder-Mead: ", opt.minimize(Log_Lik, params, args=(data), method='Nelder-Mead', bounds=bounds).x)
print("1971; Nelder-Mead: ",opt.minimize(Log_Lik, params, args=(data1971), method='Nelder-Mead', bounds=bounds).x)
print("1980; Nelder-Mead: ", opt.minimize(Log_Lik, params, args=(data1980), method='Nelder-Mead', bounds=bounds).x)
print("1990; Nelder-Mead: ", opt.minimize(Log_Lik, params, args=(data1990), method='Nelder-Mead', bounds=bounds).x)
print("2000; Nelder-Mead: ", opt.minimize(Log_Lik, params, args=(data2000), method='Nelder-Mead', bounds=bounds).x)
print("____________________________________________")
print("Whole Data; L-BFGS-B: ", opt.minimize(Log_Lik, params, args=(data), method='L-BFGS-B', bounds=bounds).x)
print("1971; L-BFGS-B: ", opt.minimize(Log_Lik, params, args=(data1971), method='L-BFGS-B', bounds=bounds).x)
print("1980; L-BFGS-B: ", opt.minimize(Log_Lik, params, args=(data1980), method='L-BFGS-B', bounds=bounds).x)
print("1990; L-BFGS-B: ", opt.minimize(Log_Lik, params, args=(data1990), method='L-BFGS-B', bounds=bounds).x)
print("2000; L-BFGS-B: ", opt.minimize(Log_Lik, params, args=(data2000), method='L-BFGS-B', bounds=bounds).x)
print("____________________________________________")
print("Whole Data; SLSQP: ",opt.minimize(Log_Lik, params, args=(data), method='SLSQP', bounds=bounds).x)
print("1971; L-BFGS-B: ", opt.minimize(Log_Lik, params, args=(data1971), method='SLSQP', bounds=bounds).x)
print("1980; L-BFGS-B: ", opt.minimize(Log_Lik, params, args=(data1980), method='SLSQP', bounds=bounds).x)
print("1990; L-BFGS-B: ", opt.minimize(Log_Lik, params, args=(data1990), method='SLSQP', bounds=bounds).x)
print("2000; L-BFGS-B: ", opt.minimize(Log_Lik, params, args=(data2000), method='SLSQP', bounds=bounds).x)

Whole Data; Nelder-Mead:  [ 0.52884269  0.07643427  0.06030063 -0.00055764 -0.1622633   0.02238691
  0.48715785]
1971; Nelder-Mead:  [ 0.58834311  0.06654214  0.0647419  -0.0006149  -0.16343732  0.03409593
  0.40586102]
1980; Nelder-Mead:  [ 1.00226977e+00  6.60040202e-02  4.55685630e-02 -3.99414608e-04
 -1.03030994e-01  1.11299464e-02  4.47797493e-01]
1990; Nelder-Mead:  [ 0.27171988  0.09555082  0.05804516 -0.00054251 -0.16514559  0.03978249
  0.48162663]
2000; Nelder-Mead:  [-0.27422273  0.10983469  0.0834132  -0.000874   -0.25305129  0.05656016
  0.5345765 ]
____________________________________________
Whole Data; L-BFGS-B:  [ 5.91257715e-01  7.60834997e-02  5.74281077e-02 -5.23691503e-04
 -1.63222088e-01  1.81296224e-02 -4.87162777e-01]
1971; L-BFGS-B:  [ 0.59436348  0.06647006  0.06450692 -0.00061222 -0.16389354  0.02550106
  0.4058383 ]
1980; L-BFGS-B:  [ 6.17745409e-01  6.79124421e-02  6.36838042e-02 -6.13345643e-04
 -9.84533954e-02  1.86309288e-02  4.48282534e-01]
1990; L-BFGS

In [79]:
# OLS 
print('OLS; Whole data',  statmod.OLS(endog=data['ln_hrw'], exog=data[['constant', 'hyrsed', 'age','age squared', 'Black', 'Others']]).fit().summary())
print('OLS; year = 1971', statmod.OLS(endog=data1971['ln_hrw'], exog=data1971[['constant', 'hyrsed', 'age','age squared', 'Black', 'Others']]).fit().summary())
print('OLS; year = 1980', statmod.OLS(endog=data1980['ln_hrw'], exog=data1980[['constant', 'hyrsed', 'age','age squared', 'Black', 'Others']]).fit().summary())
print('OLS; year = 1990', statmod.OLS(endog=data1990['ln_hrw'], exog=data1990[['constant', 'hyrsed', 'age','age squared', 'Black', 'Others']]).fit().summary())
print('OLS; year = 2000', statmod.OLS(endog=data2000['ln_hrw'], exog=data2000[['constant', 'hyrsed', 'age','age squared', 'Black', 'Others']]).fit().summary())

OLS; Whole data                             OLS Regression Results                            
Dep. Variable:                 ln_hrw   R-squared:                       0.198
Model:                            OLS   Adj. R-squared:                  0.198
Method:                 Least Squares   F-statistic:                     2824.
Date:                Mon, 04 Oct 2021   Prob (F-statistic):               0.00
Time:                        22:52:13   Log-Likelihood:                -39929.
No. Observations:               57062   AIC:                         7.987e+04
Df Residuals:                   57056   BIC:                         7.992e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
constant        0.5286      0.038 

# Interpretation
The coefficients of education in all the above estimations are positive and progressive from 1971 to 2000 based. As the desired model is linear, the results for OLS estimation and MLE are very closed and  same in magnitude and sign. That is, one additional year of education for male heads increases their wages by 7.6% in all estimations for the whole sample. The results for the other samples of the data (4 years of 1971, 1980, 1990, and 2000) are also very close to the OLS estimation in these samples.